In [1]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="https://91237092-a624-4e77-b79e-ea5796828738.us-east4-0.gcp.cloud.qdrant.io:6333", 
    api_key="6fr33Joo1ZTwrDgoPIIjWvo5dNvUfeMNfjCj79sbIx3Aq5gSJ7952g",
)

print(qdrant_client.get_collections())

collections=[]


In [7]:
import os

base_directory = "../comic_series"
comic_series = os.listdir(base_directory)

In [8]:
series_to_image= []
for series in comic_series:
    if series == ".DS_Store":
        continue
    series_path = os.path.join(base_directory, series)
    series_images = os.listdir(series_path)
    for image in series_images:
        if image == ".DS_Store":
            continue
        image_path = os.path.join(base_directory, series, image)
        series_to_image.append((series, image_path))

In [9]:
from pandas import DataFrame
from PIL import Image

payloads = DataFrame.from_records(series_to_image, columns=["series", "image"])


In [12]:
images = [Image.open(image) for image in payloads["image"]]

In [10]:
from io import BytesIO
import math
import base64

def convert_image_to_base64(image_path):
    pil_image = Image.open(image_path)
    image_data = BytesIO()
    pil_image.save(image_data, format="JPEG")
    base64_string = base64.b64encode(image_data.getvalue()).decode("utf-8")
    return base64_string

base64_strings = [convert_image_to_base64(image_path) for image_path in payloads["image"]]
payloads["base64"] = base64_strings

In [ ]:
from transformers import AutoImageProcessor, ResNetForImageClassification

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

inputs = processor(images, return_tensors="pt")
outputs = model(**inputs)
embeddings = outputs.logits

In [ ]:
embeddings.shape # 14.408 mb